In [1]:
#imports
import plotly as py
import plotly.tools as tls
from plotly import graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
from IPython.display import Image

import dash
from dash.dependencies import Output, Input
import dash_core_components as dcc
import dash_html_components as html

import numpy as np
import pandas as pd
import ipywidgets as widgets
import time
import math

from scipy import special
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot



import random
import datetime
import os
import math
from ast import literal_eval
from PIL import Image

In [2]:
#Plotly offline mode
init_notebook_mode(connected=True)
app = dash.Dash(__name__)

In [3]:
#Function to check if any of the items contain an array
def array_check(item,timer):
    item = item[0]
    if(isinstance(item, str)):
        item = literal_eval(item)
        return round(item[timer],2)
    else:
        return round(float(item),2)

In [4]:
def graph_figure(robots,timer,code):
    #initialising global variables
    

    position_robot = robots[:,:,timer]
    x_max = max(robots[0,:,timer])
    x_mag = math.floor(math.log(x_max,10))
    y_max = max(robots[1,:,timer])
    y_mag = math.floor(math.log(y_max,10))
    if(x_max and y_max > 1000):
        x_max = round(x_max,-x_mag)
        y_max = round(x_max,-y_mag)
    else:
        x_max = 1000
        y_max = 1000

    #creating scatter plot of robots
    data = go.Scatter(
        x=list(position_robot[0]),
        y=list(position_robot[1]),
        name = 'Robots',
        mode = 'markers',
    )
    
    #creating the plotly figure with the robot data
    fig = go.Figure(
        { "data": data , "layout": go.Layout(yaxis=dict(range=[-y_max, y_max]),xaxis = dict(range=[-x_max,x_max]))
        }
    )

    #updating layout with circles and different formatting'''
    fig.update_layout(title="<b>Physics Based Swarm Experiment "+ code + "</b>",
    title_x=0.5,
    xaxis_title="X Position (m)",
    yaxis_title="Y Position (m)",
    margin=dict(
        t=50, # top margin: 30px, you want to leave around 30 pixels to
              # display the modebar above the graph.
         # bottom margin: 10px
        l=10, # left margin: 10px
        r=10, # right margin: 10px
    ),

#     height=900,width=1150,
     xaxis = dict(
         tickmode = 'linear',
         tick0 = 0,
         dtick = x_max/10
     ),
                       yaxis = dict(
         tickmode = 'linear',
         tick0 = 0,
         dtick = y_max/10
     )
                     )
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black',mirror=True)
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black',mirror=True)
    
    return fig

In [5]:
#Function to check if any of the items contain an array
def array_check(item,timer):
    item = item[0]
    if(isinstance(item, str)):
        item = literal_eval(item)
        return round(item[timer],2 - int(math.floor(math.log10(abs(item[timer])))) - 1)
    else:
        return round(float(item),2 - int(math.floor(math.log10(abs(item)))) - 1)

In [6]:
def performance_graph(min_neighbours,code):
    #initialising global variables
    

    
    #creating scatter plot of robots
    data = go.Scatter(
        x=tuple(range(len(min_neighbours))),
        y=min_neighbours,
        mode = 'lines+markers',
    )
    
    #creating the plotly figure with the robot data
    fig = go.Figure(
        { "data": data
        }
    )

    #updating layout with circles and different formatting'''
    fig.update_layout(title="<b>Minimum Average Neighbour Distance (m) versus Time (s) for Experiment "+ code + "</b>",
    title_x=0.5,
    xaxis_title="Time (s)",
    yaxis_title="Minimum Average Neighbour Distance (m) ",
    margin=dict(
        t=50, # top margin: 30px, you want to leave around 30 pixels to
              # display the modebar above the graph.
         # bottom margin: 10px
        l=10, # left margin: 10px
        r=10, # right margin: 10px
    ),
    height=900,width=1150,
                      yaxis = {'range': [0, math.ceil(np.max(min_neighbours)+1)],}
                            
    
                     )
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black',mirror=True)
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black',mirror=True)
    
    return fig

In [7]:
def derivative_graph(min_neighbours,code):
    step = 5
    min_neighbours = min_neighbours[::step]
    
    x_neighbours = np.asarray(tuple(range(len(min_neighbours))))*step
    y_neighbours = min_neighbours
    
    dy = np.zeros(len(min_neighbours))
    dy[0:-1] = np.diff(y_neighbours)/np.diff(x_neighbours)
    dy[-1] = (y_neighbours[-1] - y_neighbours[-2])/(x_neighbours[-1] - x_neighbours[-2])
    print(np.where(dy == 0)[0])
    #creating scatter plot of robots
    data = go.Scatter(
        x=x_neighbours,
        y=dy,
        mode = 'lines+markers',
    )
    
    #creating the plotly figure with the robot data
    fig = go.Figure(
        { "data": data,
            }
    )

    #updating layout with circles and different formatting'''
    fig.update_layout(title="<b>Derivative of Closest Distance to Neighbour Average (m) versus Time (s) for Experiment "+ code + "</b>",
    title_x=0.5,
    xaxis_title="Time (s)",
    yaxis_title="Minimum Average Neighbour Distance (m) ",
    margin=dict(
        t=50, # top margin: 30px, you want to leave around 30 pixels to
              # display the modebar above the graph.
         # bottom margin: 10px
        l=10, # left margin: 10px
        r=10, # right margin: 10px
    ),
    height=900,width=1150,
#     xaxis = dict(
#          tickmode = 'linear',
#          tick0 = 0,
#          dtick = 0.1
#      ),
#     yaxis = dict(
#          tickmode = 'linear',
#          tick0 = 0,
#          dtick = 1
#      )
                            
    
                     )
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black',mirror=True)
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black',mirror=True)
    
    return fig

In [8]:
def table_figure(robots,timer):
    fig = go.Figure(data=[go.Table(header=dict(values=['<b>Simulation Parameters</b>','<b>Values</b>'],
                            line_color='black',
                            font=dict(color='black', size=17)),
                            cells=dict(values=[[ '<b>Simulation Timestep (h:min:s)</b>','<b>Number of Robots</b>','<b>Communication Range (m)</b>', '<b>Gravitational Constant</b>','<b>Power</b>','<b>Local Multiplier</b>','<b>Max Force (N)</b>', '<b>Max Speed (m/s)</b>', '<b>Minimum Neighbour Average (m)</b>', ], [str(datetime.timedelta(seconds=timer)),len(robots[0,:,timer]),array_check(constants.loc["Sensor Range"].values,timer),array_check(constants.loc["G"].values,timer),array_check(constants.loc["Power"].values,timer),array_check(constants.loc["Multiplier"].values,timer),array_check(constants.loc["Max Force"].values,timer),array_check(constants.loc["Max Speed"].values,timer),array_check(constants.loc["Minimum Distance to Neighbours"].values,timer)]],align='center',line_color='black'))])
    # fig.layout['template']['data']['table'][0]['header']['fill']['color']='#636EFA'
    fig.update_layout(width = 650, height = 700,margin=dict(
t=260, # top margin: 30px, you want to leave around 30 pixels to
              # display the modebar above the graph.
        b=10, # bottom margin: 10px
        l=10, # left margin: 10px
        r=10,
Jupyter Notebook
Post_Processing Last Checkpoint: 18/12/2020 (unsaved changes) Current Kernel Logo 

Python 3

    File
    Edit
    View
    Insert
    Cell
    Kernel
    Widgets
    Help

#imports

import plotly as py

import plotly.tools as tls

from plotly import graph_objs as go

import plotly.express as px

from plotly.subplots import make_subplots

from IPython.display import Image

​

import dash

from dash.dependencies import Output, Input

import dash_core_components as dcc

import dash_html_components as html

​

import numpy as np

import pandas as pd

import ipywidgets as widgets

import time

import math

​

from scipy import special

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

​

​

​

import random

import datetime

import os

import math

from ast import literal_eval

from PIL import Image

#Plotly offline mode

init_notebook_mode(connected=True)

app = dash.Dash(__name__)

#Function to check if any of the items contain an array

def array_check(item,timer):

    item = item[0]

    if(isinstance(item, str)):

        item = literal_eval(item)

        return round(item[timer],2)

    else:

        return round(float(item),2)

def graph_figure(robots,timer,code):

    #initialising global variables

    

​

    position_robot = robots[:,:,timer]

    x_max = max(robots[0,:,timer])

    x_mag = math.floor(math.log(x_max,10))

    y_max = max(robots[1,:,timer])

    y_mag = math.floor(math.log(y_max,10))

    if(x_max and y_max > 1000):

        x_max = round(x_max,-x_mag)

        y_max = round(x_max,-y_mag)

    else:

        x_max = 1000

        y_max = 1000

​

    #creating scatter plot of robots

    data = go.Scatter(

        x=list(position_robot[0]),

        y=list(position_robot[1]),

        name = 'Robots',

        mode = 'markers',

    )

    

    #creating the plotly figure with the robot data

    fig = go.Figure(

        { "data": data , "layout": go.Layout(yaxis=dict(range=[-y_max, y_max]),xaxis = dict(range=[-x_max,x_max]))

        }

    )

​

    #updating layout with circles and different formatting'''

    fig.update_layout(title="<b>Physics Based Swarm Experiment "+ code + "</b>",

    title_x=0.5,

    xaxis_title="X Position (m)",

    yaxis_title="Y Position (m)",

    margin=dict(

        t=50, # top margin: 30px, you want to leave around 30 pixels to

              # display the modebar above the graph.

         # bottom margin: 10px

        l=10, # left margin: 10px

        r=10, # right margin: 10px

    ),

​

#     height=900,width=1150,

     xaxis = dict(

         tickmode = 'linear',

         tick0 = 0,

         dtick = x_max/10

     ),

                       yaxis = dict(

         tickmode = 'linear',

         tick0 = 0,

         dtick = y_max/10

     )

                     )

    fig.update_xaxes(showline=True, linewidth=2, linecolor='black',mirror=True)

    fig.update_yaxes(showline=True, linewidth=2, linecolor='black',mirror=True)

    

    return fig

#Function to check if any of the items contain an array

def array_check(item,timer):

    item = item[0]

    if(isinstance(item, str)):

        item = literal_eval(item)

        return round(item[timer],2 - int(math.floor(math.log10(abs(item[timer])))) - 1)

    else:

        return round(float(item),2 - int(math.floor(math.log10(abs(item)))) - 1)

def performance_graph(min_neighbours,code):

    #initialising global variables

    

​

    

    #creating scatter plot of robots

    data = go.Scatter(

        x=tuple(range(len(min_neighbours))),

        y=min_neighbours,

        mode = 'lines+markers',

    )

    

    #creating the plotly figure with the robot data

    fig = go.Figure(

        { "data": data

        }

    )

​

    #updating layout with circles and different formatting'''

    fig.update_layout(title="<b>Minimum Average Neighbour Distance (m) versus Time (s) for Experiment "+ code + "</b>",

    title_x=0.5,

    xaxis_title="Time (s)",

    yaxis_title="Minimum Average Neighbour Distance (m) ",

    margin=dict(

        t=50, # top margin: 30px, you want to leave around 30 pixels to

              # display the modebar above the graph.

         # bottom margin: 10px

        l=10, # left margin: 10px

        r=10, # right margin: 10px

    ),

    height=900,width=1150,

                      yaxis = {'range': [0, math.ceil(np.max(min_neighbours)+1)],}

                            

    

                     )

    fig.update_xaxes(showline=True, linewidth=2, linecolor='black',mirror=True)

    fig.update_yaxes(showline=True, linewidth=2, linecolor='black',mirror=True)

    

    return fig

def derivative_graph(min_neighbours,code):

    step = 5

    min_neighbours = min_neighbours[::step]

    

    x_neighbours = np.asarray(tuple(range(len(min_neighbours))))*step

    y_neighbours = min_neighbours

    

    dy = np.zeros(len(min_neighbours))

    dy[0:-1] = np.diff(y_neighbours)/np.diff(x_neighbours)

    dy[-1] = (y_neighbours[-1] - y_neighbours[-2])/(x_neighbours[-1] - x_neighbours[-2])

    print(np.where(dy == 0)[0])

    #creating scatter plot of robots

    data = go.Scatter(

        x=x_neighbours,

        y=dy,

        mode = 'lines+markers',

    )

    

    #creating the plotly figure with the robot data

    fig = go.Figure(

        { "data": data,

            }

    )

​

    #updating layout with circles and different formatting'''

    fig.update_layout(title="<b>Derivative of Closest Distance to Neighbour Average (m) versus Time (s) for Experiment "+ code + "</b>",

    title_x=0.5,

    xaxis_title="Time (s)",

    yaxis_title="Minimum Average Neighbour Distance (m) ",

    margin=dict(

        t=50, # top margin: 30px, you want to leave around 30 pixels to

              # display the modebar above the graph.

         # bottom margin: 10px

        l=10, # left margin: 10px

        r=10, # right margin: 10px

    ),

    height=900,width=1150,

#     xaxis = dict(

#          tickmode = 'linear',

#          tick0 = 0,

#          dtick = 0.1

#      ),

#     yaxis = dict(

#          tickmode = 'linear',

#          tick0 = 0,

#          dtick = 1

#      )

                            

    

                     )

    fig.update_xaxes(showline=True, linewidth=2, linecolor='black',mirror=True)

    fig.update_yaxes(showline=True, linewidth=2, linecolor='black',mirror=True)

    

    return fig

def table_figure(robots,timer):

    fig = go.Figure(data=[go.Table(header=dict(values=['<b>Simulation Parameters</b>','<b>Values</b>'],

                            line_color='black',

                            font=dict(color='black', size=17)),

                            cells=dict(values=[[ '<b>Simulation Timestep (h:min:s)</b>','<b>Number of Robots</b>','<b>Communication Range (m)</b>', '<b>Gravitational Constant</b>','<b>Power</b>','<b>Local Multiplier</b>','<b>Max Force (N)</b>', '<b>Max Speed (m/s)</b>', '<b>Minimum Neighbour Average (m)</b>', ], [str(datetime.timedelta(seconds=timer)),len(robots[0,:,timer]),array_check(constants.loc["Sensor Range"].values,timer),array_check(constants.loc["G"].values,timer),array_check(constants.loc["Power"].values,timer),array_check(constants.loc["Multiplier"].values,timer),array_check(constants.loc["Max Force"].values,timer),array_check(constants.loc["Max Speed"].values,timer),array_check(constants.loc["Minimum Distance to Neighbours"].values,timer)]],align='center',line_color='black'))])

    # fig.layout['template']['data']['table'][0]['header']['fill']['color']='#636EFA'

    fig.update_layout(width = 650, height = 700,margin=dict(

t=260, # top margin: 30px, you want to leave around 30 pixels to

              # display the modebar above the graph.

        b=10, # bottom margin: 10px

        l=10, # left margin: 10px

        r=10,

    ))

    fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)

    fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True)

    return fig
    ))
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
    return fig

In [9]:
#function to combine the table and graph images
def combine(graph_path,table_path):
    image_graph = Image.open(graph_path)
    image_table = Image.open(table_path)
    new1 = Image.new('RGB', (image_graph.width + image_table.width, min(image_graph.height,image_table.height)))
    new1.paste(image_graph, (0, 0))
    new1.paste(image_table, (image_graph.width,((image_graph.height-image_table.height)//2)-86))
    new1.save(graph_path)
    os.remove(table_path)

In [10]:
G_array = [10**-10,10**-5,10**-2,10**-1,10**0,10**1,10**2,10**3,10**4,10**5,10**6]
power_array = [2,3,4,5]
R_array = 500#[10,50,100,125,150,200,250,500]
multiply_array = [1,1.1,1.2,1.3,1.4,1.5,2]
robot_number_array = [500]

# G_array = [10000]
# power_array = [2]
# R_array = [150]
# multiply_array = [1.5]
# robot_number_array = [100]


#setting timestep parameter
time = 1000

current_directory = "Experiments/Removed Bias/"

try:
    os.mkdir(current_directory)
except OSError:
    print ("Failure: Directory creation of %s failed" % current_directory)
else:
    print ("Success: Directory creation of %s succeeded" % current_directory)

for robot in robot_number_array:
    
    robot_path = (current_directory + str(robot) + " Robots/")
    try:
        os.mkdir(robot_path)
    except OSError:
        print ("Failure: Directory creation of %s failed" % robot_path)
    else:
        print ("Success: Directory creation of %s succeeded" % robot_path)
    image = (current_directory + str(robot) + " Robots/Images/")
    try:
        os.mkdir(image)
    except OSError:
        print ("Failure: Directory creation of %s failed" % robot_path)
    else:
        print ("Success: Directory creation of %s succeeded" % robot_path)
    for i in range(len(G_array)):
        
        #Experiment number
        code = "G" + str(i)
  
        robots = (current_directory + str(robot) + " Robots/")
        path = (current_directory + str(robot) + " Robots/" + code + "/")
        print(path)
        try:
            os.mkdir(path)
        except OSError:
            print ("Creation of the directory %s failed" % path)
        else:
            print ("Successfully created the directory %s " % path)
        
       
        store_robots = np.load(path + "Robots.npy")
        
        constants = pd.read_excel(path + "Constants.xlsx", index_col=0)

        temp_neighbours = constants.loc['Minimum Distance to Neighbours']
        min_neighbours = literal_eval(temp_neighbours[0])
        
        

        image = (current_directory + str(robot) + " Robots/Images/")
        
        graph_start_path = image + code + "- Graph_Beginning.png"
        table_start_path = image + "/" + code + "- Table_Beginning.png"
        
        graph_start = graph_figure(store_robots,0,code)
        graph_start.write_image(graph_start_path)
        table_start = table_figure(store_robots,0)
        table_start.write_image(table_start_path)
        
        combine(graph_start_path,table_start_path)

        graph_end_path = image + code + "- Graph_End.png"
        table_end_path = image + "/" + code + "- Table_End.png"
        
        graph_end = graph_figure(store_robots,time,code)
        graph_end.write_image(image + "/" + code + "- Graph_End.png")
        table_end = table_figure(store_robots,time)
        table_end.write_image(image + "/" + code + "- Table_End.png")

        combine(graph_end_path,table_end_path)
        
        performance = performance_graph(min_neighbours,code)
        performance.write_image(image + "/" + code + "- Performance.png")
        
        derivative = derivative_graph(min_neighbours,code)
        derivative.write_image(image + "/" + code + "- Derivative.png")
        

Failure: Directory creation of Experiments/Removed Bias/ failed
Failure: Directory creation of Experiments/Removed Bias/500 Robots/ failed
Failure: Directory creation of Experiments/Removed Bias/500 Robots/ failed
Experiments/Removed Bias/500 Robots/G0/
Creation of the directory Experiments/Removed Bias/500 Robots/G0/ failed
[]
Experiments/Removed Bias/500 Robots/G1/
Creation of the directory Experiments/Removed Bias/500 Robots/G1/ failed
[]
Experiments/Removed Bias/500 Robots/G2/
Creation of the directory Experiments/Removed Bias/500 Robots/G2/ failed
[]
Experiments/Removed Bias/500 Robots/G3/
Creation of the directory Experiments/Removed Bias/500 Robots/G3/ failed
[]
Experiments/Removed Bias/500 Robots/G4/
Creation of the directory Experiments/Removed Bias/500 Robots/G4/ failed
[]
Experiments/Removed Bias/500 Robots/G5/
Creation of the directory Experiments/Removed Bias/500 Robots/G5/ failed
[]
Experiments/Removed Bias/500 Robots/G6/
Creation of the directory Experiments/Removed Bias